# train data에 '일반 대화' 추가
- 결과파일명: custom_train.csv

In [2]:
import pandas as pd

base_data_path = '/aiffel/aiffel/workspace/16_dlthon_1/data/'

train = pd.read_csv(base_data_path + 'train.csv')
train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [2]:
print('전체 갯수',len(train))
print('class 종류:', *set(train['class']))
print('class 협박 대화', len(train[train['class'] == '협박 대화']))
print('class 갈취 대화', len(train[train['class'] == '갈취 대화']))
print('class 직장 내 괴롭힘 대화', len(train[train['class'] == '직장 내 괴롭힘 대화']))
print('class 기타 괴롭힘 대화', len(train[train['class'] == '기타 괴롭힘 대화']))
len(train['conversation'][0])
train['conversation'].apply(len).describe()

전체 갯수 3950
class 종류: 직장 내 괴롭힘 대화 협박 대화 갈취 대화 기타 괴롭힘 대화
class 협박 대화 896
class 갈취 대화 981
class 직장 내 괴롭힘 대화 979
class 기타 괴롭힘 대화 1094


count    3950.000000
mean      226.570886
std       104.560310
min        41.000000
25%       156.000000
50%       203.000000
75%       270.000000
max       874.000000
Name: conversation, dtype: float64

In [3]:
import os

# /aiffel/aiffel/workspace/16_dlthon_1/data/normal_conversation

dirs_normal_text = ['./data/normal_conversation/02_label_data/session2/',
                    './data/normal_conversation/02_label_data/session3/',
                     './data/normal_conversation/02_label_data/session4/']

total_path = []
for dirs in dirs_normal_text:
    for file_path in os.listdir(dirs):
        total_path.append(dirs+file_path)
        
# total path 확인용
# for path in total_path:
#     print(path)

In [4]:
import json

results = []

for path in total_path:
    
    with open(path, 'r') as f:
        json_data = json.load(f)

        for session in json_data['sessionInfo']:

            tmp_text = ''
            for log in session['dialog']:
                tmp_text += log['utterance']

                # 평균 길이보다 크면 break
                if len(tmp_text) >= 226:
                    results.append(tmp_text)
                    tmp_text = ''
                    break
 
print(len(results))
print(results[3])


1100
안녕하세요 5주만에 만나서 반가워요네 반가워요. 요즘 영월에서 전통 민속놀이를 하고 있는데, 보신적 있나요?무슨민속놀이 하는데요 궁금해요단종의 죽음을 기리는 민속놀이라고 합니다.아 슬픈거내요 전 생소해서 그런건슬프긴 하지만 한 번쯤 가서 볼만 합니다. 보러 가세요.민속놀이는 투구던지기 알아요?투구던지기 알죠. 잘 던지시나요?잘던져요 저는 소질이 있나봐요최고 잘 던진 기록은 얼마인가요?백미터정도요 ㅋㅋ 농담이고요 


In [5]:
results_df = pd.DataFrame(
    {
        'idx': range(3950, 3950+len(results)),
        'class': ['일반 대화']*len(results),
        'conversation':results
    }
)

train = pd.concat([train, results_df], ignore_index=True)

In [6]:
# 확인용
print('전체 갯수',len(train))
print('class 종류:', *set(train['class']))
print('class 협박 대화', len(train[train['class'] == '협박 대화']))
print('class 갈취 대화', len(train[train['class'] == '갈취 대화']))
print('class 직장 내 괴롭힘 대화', len(train[train['class'] == '직장 내 괴롭힘 대화']))
print('class 기타 괴롭힘 대화', len(train[train['class'] == '기타 괴롭힘 대화']))
len(train['conversation'][0])
train['conversation'].apply(len).describe()

전체 갯수 5050
class 종류: 기타 괴롭힘 대화 직장 내 괴롭힘 대화 일반 대화 협박 대화 갈취 대화
class 협박 대화 896
class 갈취 대화 981
class 직장 내 괴롭힘 대화 979
class 기타 괴롭힘 대화 1094


count    5050.000000
mean      230.886535
std        93.135962
min        41.000000
25%       169.250000
50%       229.000000
75%       261.000000
max       874.000000
Name: conversation, dtype: float64

In [7]:
train.to_csv('./data/custom_train.csv', index=False)

# test 데이터 csv로
- 결과파일명 : custom_test.csv

In [21]:
import json

with open(base_data_path+'test.json', 'r') as read_file:
    org_datas = json.load(read_file)

test_datas = []

for key in org_datas:
    test_datas.append(org_datas[key]['text'])

test = pd.DataFrame(test_datas,columns=['conversation'])

test.to_csv('./data/custom_test.csv', index=False)